In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
batch_size = 32
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(train_data, batch_size=batch_size)

In [7]:
for X,y in train_dataloader:
    print("shape of X {}".format(X.shape))
    print("shape of y {}".format(y.shape))
    break

shape of X torch.Size([32, 1, 28, 28])
shape of y torch.Size([32])


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        

In [9]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(train_dataloader.dataset)
    model.train()
    
    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader,model, loss_fn):
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    
    model.eval()
    test_loss, correct = 0,0
    
    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss+=loss_fn(pred, y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=num_batch
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [16]:
epoch = 5

for t in range(epoch):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
loss: 1.867326  [    0/60000]
loss: 1.890507  [ 3200/60000]
loss: 1.812695  [ 6400/60000]
loss: 1.772940  [ 9600/60000]
loss: 1.696552  [12800/60000]
loss: 1.735273  [16000/60000]
loss: 1.583814  [19200/60000]
loss: 1.571677  [22400/60000]
loss: 1.554009  [25600/60000]
loss: 1.520257  [28800/60000]
loss: 1.398094  [32000/60000]
loss: 1.326177  [35200/60000]
loss: 1.532753  [38400/60000]
loss: 1.396253  [41600/60000]
loss: 1.311266  [44800/60000]
loss: 1.265163  [48000/60000]
loss: 1.371362  [51200/60000]
loss: 1.369852  [54400/60000]
loss: 1.186683  [57600/60000]
Test Error: 
 Accuracy: 63.6%, Avg loss: 1.235992 

Epoch 2
-------------------------------
loss: 1.239602  [    0/60000]
loss: 1.239381  [ 3200/60000]
loss: 1.233616  [ 6400/60000]
loss: 1.288335  [ 9600/60000]
loss: 1.130945  [12800/60000]
loss: 1.284303  [16000/60000]
loss: 1.008207  [19200/60000]
loss: 1.039525  [22400/60000]
loss: 1.086057  [25600/60000]
loss: 1.166634  [28800/60000

In [17]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [18]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
